In [1]:
from __future__ import absolute_import, division, print_function
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from numpy  import array
from scipy import stats
from gensim.models.callbacks import CallbackAny2Vec

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# PREPROCESSING

In [3]:
corpus_filename = '../data/extracted_emoji_sequences.txt'

In [4]:
corpus = open(corpus_filename).read().splitlines()


In [5]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [6]:
def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

In [7]:
tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

In [8]:
token_count = sum([len(sentence) for sentence in emojiSequences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 610,256 tokens


# SETTING VARIABLES

In [9]:
# Dimensionality of the resulting word vectors.
num_features = 300

# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 2

# Downsample setting for frequent words.
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
# remove later
seed = 1

emoji2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    def __init__(self):
        self.epoch = 0
    def on_epoch_end(self, model):
        print("Epoch #{}  Loss-Value: {}".format(self.epoch, emoji2vec.get_latest_training_loss()))
        self.epoch += 1
        
epoch_logger = EpochLogger()

emoji2vec.build_vocab(emojiSequences, progress_per=10000000)

# TRAINING

In [10]:
emoji2vec.train(emojiSequences, total_examples=emoji2vec.corpus_count, epochs = 50, compute_loss=True, callbacks=[epoch_logger])

Epoch #0 . Loss-Value: 142701.515625
Epoch #1 . Loss-Value: 227239.5625
Epoch #2 . Loss-Value: 304366.78125
Epoch #3 . Loss-Value: 388530.40625
Epoch #4 . Loss-Value: 458362.96875
Epoch #5 . Loss-Value: 534406.375
Epoch #6 . Loss-Value: 606288.25
Epoch #7 . Loss-Value: 686856.4375
Epoch #8 . Loss-Value: 749028.125
Epoch #9 . Loss-Value: 827810.875
Epoch #10 . Loss-Value: 894664.0
Epoch #11 . Loss-Value: 961246.5
Epoch #12 . Loss-Value: 1035710.1875
Epoch #13 . Loss-Value: 1102007.25
Epoch #14 . Loss-Value: 1165470.125
Epoch #15 . Loss-Value: 1238970.625
Epoch #16 . Loss-Value: 1310930.625
Epoch #17 . Loss-Value: 1386424.25
Epoch #18 . Loss-Value: 1457585.5
Epoch #19 . Loss-Value: 1525235.75
Epoch #20 . Loss-Value: 1597838.875
Epoch #21 . Loss-Value: 1660244.625
Epoch #22 . Loss-Value: 1722467.375
Epoch #23 . Loss-Value: 1784240.375
Epoch #24 . Loss-Value: 1846862.125
Epoch #25 . Loss-Value: 1919028.625
Epoch #26 . Loss-Value: 1979662.5
Epoch #27 . Loss-Value: 2050654.625
Epoch #28 . Lo

(19443895, 30512800)

In [11]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [13]:
emoji2vec.save(os.path.join("trained", "2nd.w2v"))

# EVALUATING TRAINED MODEL

In [14]:
emoji2vec = w2v.Word2Vec.load(os.path.join("trained", "2nd.w2v"))

In [15]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = emoji2vec.wv.similarity(emoji1, emoji2)
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        selftrained.append(cosineSimilarity)
        goldstandard.append(annotator_similarity_score_508.pop(0))

# computation of SPEARRANK CORRELATION COEFFICIENT
spearmanRank = stats.spearmanr(goldstandard, selftrained)

print('Der Spearman Rank Correlation Coefficient is {}'.format(spearmanRank))

the cosine similarity between 🇬🇧 and 🇺🇸 could not be computed.
the cosine similarity between 🏅 and 🇺🇸 could not be computed.
the cosine similarity between 🇺🇸 and ❤ could not be computed.
the cosine similarity between 🇺🇸 and 💥 could not be computed.
the cosine similarity between 🎤 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 📲 could not be computed.
the cosine similarity between 👇 and 🇳🇬 could not be computed.
the cosine similarity between 🎧 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 🎶 could not be computed.
the cosine similarity between 👏 and ↪ could not be computed.
Der Spearman Rank Correlation Coefficient is SpearmanrResult(correlation=-0.01886830858449247, pvalue=0.6744541653251483)


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [19]:
emoji2vec.wv.most_similar('😟')



/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('😑', 0.5965372323989868),
 ('😧', 0.5478870868682861),
 ('😦', 0.5450003147125244),
 ('😖', 0.5350342988967896),
 ('😷', 0.5086485743522644),
 ('😵', 0.4946061968803406),
 ('😕', 0.4879322052001953),
 ('🤒', 0.4814000427722931),
 ('😣', 0.46933263540267944),
 ('🙁', 0.46830254793167114)]